In [1]:
import numpy as np
import pandas as pd
from visits import Visit, Cohort, MissingDataError, holm_bonferroni, rolling_variance, rolling_autocorrelation
import glob
import pymannkendall as mk
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu, ttest_ind
import seaborn as sb
from matplotlib import rc


In [2]:
cohort_1_details = pd.read_csv("cohort_1_details_v5.csv", sep=',')
cohort_2_details = pd.read_csv("cohort_2_details_v5.csv", sep=',')
cohort_3_details = pd.read_csv("cohort_3_details_v5.csv", sep=',')
cohort_4_details = pd.read_csv("cohort_4_details_v5.csv", sep=',')

In [3]:
var_hr_c1_results = pd.read_csv("variance_results_c1.csv", sep=",")
var_rr_c1_results = pd.read_csv("variance_rr_results_c1.csv", sep=",")
var_abf_c1_results = pd.read_csv("variance_abf_results_c1.csv", sep=",")

ac_hr_c1_results = pd.read_csv("autocorrelation_hr_results_c1.csv", sep=",")
ac_rr_c1_results = pd.read_csv("autocorrelation_rr_results_c1.csv", sep=",")
ac_abf_c1_results = pd.read_csv("autocorrelation_abf_results_c1.csv", sep=",")

In [4]:
var_hr_c1_results

,Unnamed: 0,Project ID,visit_no,trend,p-value,tau
0,0,RD3976,1,increasing,0.000030,0.576651
1,1,RX1828,1,no trend,0.693120,-0.081616
2,2,RP0984,1,no trend,0.365649,0.201415
3,3,UI4061,1,no trend,0.870594,-0.032581
4,4,UD1688,1,increasing,0.000060,0.915406
...,...,...,...,...,...,...
107,107,SZ0828,4,no trend,0.873990,0.013677
108,108,CN1187,4,no trend,0.906997,-0.022346
109,109,DD1778,4,no trend,0.610578,0.046929
110,110,ST0725,5,no trend,0.967761,-0.003136


In [5]:
# var_hr_c1_results_tuples = pd.DataFrame(columns=["Project ID", "visit_no", "tau", "p-value"])
# var_rr_c1_results_tuples = pd.DataFrame(columns=["Project ID", "visit_no", "tau", "p-value"])
# var_abf_c1_results_tuples = pd.DataFrame(columns=["Project ID", "visit_no", "tau", "p-value"])
# ac_hr_c1_results_tuples = pd.DataFrame(columns=["Project ID", "visit_no", "tau", "p-value"])
# ac_rr_c1_results_tuples = pd.DataFrame(columns=["Project ID", "visit_no", "tau", "p-value"])
# ac_abf_c1_results_tuples = pd.DataFrame(columns=["Project ID", "visit_no", "tau", "p-value"])

In [6]:
# var_hr_c1_results_tuples["Project ID"] == 

In [7]:
# var_hr_c1_results_tuples = list(zip(var_hr_c1_results["Project ID"], var_hr_c1_results["visit_no"], var_hr_c1_results["tau"], var_hr_c1_results["p-value"]))
# # var_hr_c1_failed_tuples = list(zip(var_hr_c1_failed["Project ID"], var_hr_c1_failed["visit_no"]))

# var_rr_c1_results_tuples = list(zip(var_rr_c1_results["Project ID"], var_rr_c1_results["visit_no"], var_rr_c1_results["tau"], var_rr_c1_results["p-value"]))
# # var_rr_c1_failed_tuples = list(zip(var_rr_c1_failed["Project ID"], var_rr_c1_failed["visit_no"]))

# var_abf_c1_results_tuples = list(zip(var_abf_c1_results["Project ID"], var_abf_c1_results["visit_no"], var_abf_c1_results["tau"], var_abf_c1_results["p-value"]))
# # var_abf_c1_failed_tuples = list(zip(var_abf_c1_failed["Project ID"], var_abf_c1_failed["visit_no"]))

# ac_hr_c1_results_tuples = list(zip(ac_hr_c1_results["Project ID"], ac_hr_c1_results["visit_no"], ac_hr_c1_results["tau"], ac_hr_c1_results["p-value"]))
# # ac_hr_c1_failed_tuples = list(zip(ac_hr_c1_failed["Project ID"], ac_hr_c1_failed["visit_no"]))

# ac_rr_c1_results_tuples = list(zip(ac_rr_c1_results["Project ID"], ac_rr_c1_results["visit_no"], ac_rr_c1_results["tau"], ac_rr_c1_results["p-value"]))
# # ac_rr_c1_failed_tuples = list(zip(ac_rr_c1_failed["Project ID"], ac_rr_c1_failed["visit_no"]))

# ac_abf_c1_results_tuples = list(zip(ac_abf_c1_results["Project ID"], ac_abf_c1_results["visit_no"], ac_abf_c1_results["tau"], ac_abf_c1_results["p-value"]))
# # ac_abf_c1_failed_tuples = list(zip(ac_abf_c1_failed["Project ID"], ac_abf_c1_failed["visit_no"]))

In [8]:
var_hr_c1_results_tuples = list(zip(var_hr_c1_results["Project ID"], var_hr_c1_results["visit_no"]))
# var_hr_c1_failed_tuples = list(zip(var_hr_c1_failed["Project ID"], var_hr_c1_failed["visit_no"]))

var_rr_c1_results_tuples = list(zip(var_rr_c1_results["Project ID"], var_rr_c1_results["visit_no"]))
# var_rr_c1_failed_tuples = list(zip(var_rr_c1_failed["Project ID"], var_rr_c1_failed["visit_no"]))

var_abf_c1_results_tuples = list(zip(var_abf_c1_results["Project ID"], var_abf_c1_results["visit_no"]))
# var_abf_c1_failed_tuples = list(zip(var_abf_c1_failed["Project ID"], var_abf_c1_failed["visit_no"]))

ac_hr_c1_results_tuples = list(zip(ac_hr_c1_results["Project ID"], ac_hr_c1_results["visit_no"]))
# ac_hr_c1_failed_tuples = list(zip(ac_hr_c1_failed["Project ID"], ac_hr_c1_failed["visit_no"]))

ac_rr_c1_results_tuples = list(zip(ac_rr_c1_results["Project ID"], ac_rr_c1_results["visit_no"]))
# ac_rr_c1_failed_tuples = list(zip(ac_rr_c1_failed["Project ID"], ac_rr_c1_failed["visit_no"]))

ac_abf_c1_results_tuples = list(zip(ac_abf_c1_results["Project ID"], ac_abf_c1_results["visit_no"]))
# ac_abf_c1_failed_tuples = list(zip(ac_abf_c1_failed["Project ID"], ac_abf_c1_failed["visit_no"]))

In [9]:
np.array(var_hr_c1_results_tuples)[:,0:2]


array([['RD3976', '1'],
       ['RX1828', '1'],
       ['RP0984', '1'],
       ['UI4061', '1'],
       ['UD1688', '1'],
       ['UD2443', '1'],
       ['UX1362', '1'],
       ['UB1799', '1'],
       ['UY2351', '1'],
       ['IZ1435', '1'],
       ['IZ2966', '1'],
       ['IF3856', '1'],
       ['IJ2643', '1'],
       ['NF3544', '1'],
       ['NQ3290', '1'],
       ['NX0632', '1'],
       ['NE0450', '1'],
       ['NP1006', '1'],
       ['GF0332', '1'],
       ['GF1065', '1'],
       ['GP0251', '1'],
       ['ZR0822', '1'],
       ['ZN2706', '1'],
       ['ZD2068', '1'],
       ['ZD1853', '1'],
       ['ZK0529', '1'],
       ['TR1110', '1'],
       ['TI3785', '1'],
       ['TV4037', '1'],
       ['TP3224', '1'],
       ['SZ0162', '1'],
       ['SQ1612', '1'],
       ['SE1569', '1'],
       ['SP1040', '1'],
       ['AX3634', '1'],
       ['AY3292', '1'],
       ['FN3246', '1'],
       ['FN1648', '1'],
       ['FG1244', '1'],
       ['FE3652', '1'],
       ['OS1758', '1'],
       ['OH1674'

In [10]:
cohort_1_details["var_hr_p"] = None
cohort_1_details["var_rr_p"] = None
cohort_1_details["var_abf_p"] = None
cohort_1_details["ac_hr_p"] = None
cohort_1_details["ac_rr_p"] = None
cohort_1_details["ac_abf_p"] = None

cohort_1_details["var_hr_tau"] = None
cohort_1_details["var_rr_tau"] = None
cohort_1_details["var_abf_tau"] = None
cohort_1_details["ac_hr_tau"] = None
cohort_1_details["ac_rr_tau"] = None
cohort_1_details["ac_abf_tau"] = None

cohort_1_details["var_hr_trend"] = None
cohort_1_details["var_rr_trend"] = None
cohort_1_details["var_abf_trend"] = None
cohort_1_details["ac_hr_trend"] = None
cohort_1_details["ac_rr_trend"] = None
cohort_1_details["ac_abf_trend"] = None

In [11]:
cohort_1_details

,Unnamed: 0,Project ID,birth_deid_date,death_deid_date,sex,icu_visit,icu_start_deid_dttm,icu_end_deid_dttm,failed_extubation_flag,failed_extubation_deid_date,...,var_abf_tau,ac_hr_tau,ac_rr_tau,ac_abf_tau,var_hr_trend,var_rr_trend,var_abf_trend,ac_hr_trend,ac_rr_trend,ac_abf_trend
0,85,KH0067,2003-01-22 00:00:00,NaN,Female,2,2016-10-13 22:30:00,2016-12-03 15:54:00,1.0,2016-10-16 10:30:00,...,None,None,None,None,None,None,None,None,None,None
1,165,JJ0136,2001-06-12 00:00:00,NaN,Female,2,2016-06-22 02:00:00,2016-08-22 18:08:00,1.0,2016-07-05 12:15:00,...,None,None,None,None,None,None,None,None,None,None
2,204,SZ0162,2006-01-09 00:00:00,NaN,Male,1,2016-04-12 15:30:00,2016-05-12 13:00:00,1.0,2016-04-28 16:00:00,...,None,None,None,None,None,None,None,None,None,None
3,239,PC0192,2004-06-22 00:00:00,NaN,Male,1,2016-12-22 11:53:00,2017-02-18 17:13:00,1.0,2017-01-10 18:00:00,...,None,None,None,None,None,None,None,None,None,None
4,314,GP0251,2001-10-06 00:00:00,NaN,Male,1,2018-06-25 14:44:00,2018-07-06 16:07:00,1.0,2018-06-26 15:00:00,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,5424,VV3912,2018-08-23 00:00:00,NaN,Male,2,2018-11-13 14:32:00,2018-11-17 17:20:00,1.0,2018-11-14 18:00:00,...,None,None,None,None,None,None,None,None,None,None
116,5484,PL3964,2018-09-29 00:00:00,NaN,Male,1,2018-10-10 17:30:00,2018-11-17 13:06:00,1.0,2018-10-22 15:00:00,...,None,None,None,None,None,None,None,None,None,None
117,5496,RD3976,2018-10-14 00:00:00,NaN,Male,1,2018-10-16 05:45:00,2018-10-23 19:09:00,1.0,2018-10-19 11:00:00,...,None,None,None,None,None,None,None,None,None,None
118,5565,TV4037,2018-09-28 00:00:00,NaN,Male,1,2018-12-12 20:00:00,2018-12-18 16:17:00,1.0,2018-12-13 11:50:00,...,None,None,None,None,None,None,None,None,None,None


In [12]:
x = var_hr_c1_results.loc[var_hr_c1_results["Project ID"] == "KH0067"]
x = x.loc[x["visit_no"] == 2]
x["tau"].iloc[0]

0.61270903

In [13]:
for idx, project_id, visit_no in zip(cohort_1_details.index, cohort_1_details["Project ID"], cohort_1_details["icu_visit"]):
    if (project_id, visit_no) in var_hr_c1_results_tuples:
        x = var_hr_c1_results.loc[var_hr_c1_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_1_details["var_hr_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_1_details["var_hr_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_1_details["var_hr_trend"].loc[idx] = x["trend"].iloc[0]

for idx, project_id, visit_no in zip(cohort_1_details.index, cohort_1_details["Project ID"], cohort_1_details["icu_visit"]):
    if (project_id, visit_no) in var_rr_c1_results_tuples:
        x = var_rr_c1_results.loc[var_rr_c1_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_1_details["var_rr_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_1_details["var_rr_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_1_details["var_rr_trend"].loc[idx] = x["trend"].iloc[0]

for idx, project_id, visit_no in zip(cohort_1_details.index, cohort_1_details["Project ID"], cohort_1_details["icu_visit"]):
    if (project_id, visit_no) in var_abf_c1_results_tuples:
        x = var_abf_c1_results.loc[var_abf_c1_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_1_details["var_abf_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_1_details["var_abf_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_1_details["var_abf_trend"].loc[idx] = x["trend"].iloc[0]

cohort_1_details

# cohort_1_details

# for idx, project_id, visit_no in zip(cohort_1_details.index, cohort_1_details["Project ID"], cohort_1_details["icu_visit"]):
#     if (project_id, visit_no) in var_hr_c1_failed_tuples:
#         cohort_1_details["var_hr_flag"].loc[idx] = 0

/Users/lucindakhalil/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Unnamed: 0,Project ID,birth_deid_date,death_deid_date,sex,icu_visit,icu_start_deid_dttm,icu_end_deid_dttm,failed_extubation_flag,failed_extubation_deid_date,...,var_abf_tau,ac_hr_tau,ac_rr_tau,ac_abf_tau,var_hr_trend,var_rr_trend,var_abf_trend,ac_hr_trend,ac_rr_trend,ac_abf_trend
0,85,KH0067,2003-01-22 00:00:00,NaN,Female,2,2016-10-13 22:30:00,2016-12-03 15:54:00,1.0,2016-10-16 10:30:00,...,0.048533,None,None,None,increasing,increasing,no trend,None,None,None
1,165,JJ0136,2001-06-12 00:00:00,NaN,Female,2,2016-06-22 02:00:00,2016-08-22 18:08:00,1.0,2016-07-05 12:15:00,...,-0.450952,None,None,None,no trend,decreasing,decreasing,None,None,None
2,204,SZ0162,2006-01-09 00:00:00,NaN,Male,1,2016-04-12 15:30:00,2016-05-12 13:00:00,1.0,2016-04-28 16:00:00,...,None,None,None,None,decreasing,no trend,None,None,None,None
3,239,PC0192,2004-06-22 00:00:00,NaN,Male,1,2016-12-22 11:53:00,2017-02-18 17:13:00,1.0,2017-01-10 18:00:00,...,0.296163,None,None,None,increasing,increasing,increasing,None,None,None
4,314,GP0251,2001-10-06 00:00:00,NaN,Male,1,2018-06-25 14:44:00,2018-07-06 16:07:00,1.0,2018-06-26 15:00:00,...,0.069743,None,None,None,no trend,increasing,no trend,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,5424,VV3912,2018-08-23 00:00:00,NaN,Male,2,2018-11-13 14:32:00,2018-11-17 17:20:00,1.0,2018-11-14 18:00:00,...,None,None,None,None,no trend,None,None,None,None,None
116,5484,PL3964,2018-09-29 00:00:00,NaN,Male,1,2018-10-10 17:30:00,2018-11-17 13:06:00,1.0,2018-10-22 15:00:00,...,None,None,None,None,no trend,no trend,None,None,None,None
117,5496,RD3976,2018-10-14 00:00:00,NaN,Male,1,2018-10-16 05:45:00,2018-10-23 19:09:00,1.0,2018-10-19 11:00:00,...,None,None,None,None,increasing,no trend,None,None,None,None
118,5565,TV4037,2018-09-28 00:00:00,NaN,Male,1,2018-12-12 20:00:00,2018-12-18 16:17:00,1.0,2018-12-13 11:50:00,...,None,None,None,None,no trend,no trend,None,None,None,None


In [14]:
for idx, project_id, visit_no in zip(cohort_1_details.index, cohort_1_details["Project ID"], cohort_1_details["icu_visit"]):
    if (project_id, visit_no) in ac_hr_c1_results_tuples:
        x = ac_hr_c1_results.loc[ac_hr_c1_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_1_details["ac_hr_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_1_details["ac_hr_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_1_details["ac_hr_trend"].loc[idx] = x["trend"].iloc[0]

for idx, project_id, visit_no in zip(cohort_1_details.index, cohort_1_details["Project ID"], cohort_1_details["icu_visit"]):
    if (project_id, visit_no) in ac_rr_c1_results_tuples:
        x = ac_rr_c1_results.loc[ac_rr_c1_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_1_details["ac_rr_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_1_details["ac_rr_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_1_details["ac_rr_trend"].loc[idx] = x["trend"].iloc[0]

for idx, project_id, visit_no in zip(cohort_1_details.index, cohort_1_details["Project ID"], cohort_1_details["icu_visit"]):
    if (project_id, visit_no) in ac_abf_c1_results_tuples:
        x = ac_abf_c1_results.loc[ac_abf_c1_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_1_details["ac_abf_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_1_details["ac_abf_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_1_details["ac_abf_trend"].loc[idx] = x["trend"].iloc[0]

/Users/lucindakhalil/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [15]:
cohort_1_details

,Unnamed: 0,Project ID,birth_deid_date,death_deid_date,sex,icu_visit,icu_start_deid_dttm,icu_end_deid_dttm,failed_extubation_flag,failed_extubation_deid_date,...,var_abf_tau,ac_hr_tau,ac_rr_tau,ac_abf_tau,var_hr_trend,var_rr_trend,var_abf_trend,ac_hr_trend,ac_rr_trend,ac_abf_trend
0,85,KH0067,2003-01-22 00:00:00,NaN,Female,2,2016-10-13 22:30:00,2016-12-03 15:54:00,1.0,2016-10-16 10:30:00,...,0.048533,0.427291,-0.428986,-0.341954,increasing,increasing,no trend,increasing,decreasing,decreasing
1,165,JJ0136,2001-06-12 00:00:00,NaN,Female,2,2016-06-22 02:00:00,2016-08-22 18:08:00,1.0,2016-07-05 12:15:00,...,-0.450952,-0.499802,-0.085357,0.057381,no trend,decreasing,decreasing,decreasing,no trend,no trend
2,204,SZ0162,2006-01-09 00:00:00,NaN,Male,1,2016-04-12 15:30:00,2016-05-12 13:00:00,1.0,2016-04-28 16:00:00,...,None,0.419375,-0.340171,None,decreasing,no trend,None,increasing,no trend,None
3,239,PC0192,2004-06-22 00:00:00,NaN,Male,1,2016-12-22 11:53:00,2017-02-18 17:13:00,1.0,2017-01-10 18:00:00,...,0.296163,0.336171,-0.416447,0.161712,increasing,increasing,increasing,increasing,decreasing,increasing
4,314,GP0251,2001-10-06 00:00:00,NaN,Male,1,2018-06-25 14:44:00,2018-07-06 16:07:00,1.0,2018-06-26 15:00:00,...,0.069743,0.234363,-0.241914,-0.106101,no trend,increasing,no trend,no trend,decreasing,decreasing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,5424,VV3912,2018-08-23 00:00:00,NaN,Male,2,2018-11-13 14:32:00,2018-11-17 17:20:00,1.0,2018-11-14 18:00:00,...,None,0.169468,None,None,no trend,None,None,no trend,None,None
116,5484,PL3964,2018-09-29 00:00:00,NaN,Male,1,2018-10-10 17:30:00,2018-11-17 13:06:00,1.0,2018-10-22 15:00:00,...,None,0.096265,0.076928,None,no trend,no trend,None,no trend,no trend,None
117,5496,RD3976,2018-10-14 00:00:00,NaN,Male,1,2018-10-16 05:45:00,2018-10-23 19:09:00,1.0,2018-10-19 11:00:00,...,None,0.117185,0.118876,None,increasing,no trend,None,no trend,no trend,None
118,5565,TV4037,2018-09-28 00:00:00,NaN,Male,1,2018-12-12 20:00:00,2018-12-18 16:17:00,1.0,2018-12-13 11:50:00,...,None,0.135497,-0.189592,None,no trend,no trend,None,no trend,no trend,None


In [16]:
cohort_1_details["var_hr_sig_flag"] = None
cohort_1_details["var_rr_sig_flag"] = None
cohort_1_details["var_abf_sig_flag"] = None
cohort_1_details["ac_hr_sig_flag"] = None
cohort_1_details["ac_rr_sig_flag"] = None
cohort_1_details["ac_abf_sig_flag"] = None

In [17]:
cohort_1_details.index

RangeIndex(start=0, stop=120, step=1)

In [18]:
float("{:f}".format(4e-3))

0.004

In [19]:
alpha = 0.05
passed = []

cohort_1_details_copy = cohort_1_details.copy()

for idx in cohort_1_details.index:
    row = cohort_1_details.iloc[idx]

    # temp_dict_p = {"var_hr":row["var_hr_p"], "var_rr":row["var_rr_p"], "var_abf":row["var_abf_p"], "ac_hr":row["ac_hr_p"], "ac_rr":row["ac_rr_p"], "ac_abf":row["ac_abf_p"]}

    measure_list = ["var_hr_sig_flag","var_rr_sig_flag","var_abf_sig_flag","ac_hr_sig_flag","ac_rr_sig_flag","ac_abf_sig_flag"]
    p_list = [row["var_hr_p"], row["var_rr_p"], row["var_abf_p"], row["ac_hr_p"], row["ac_rr_p"], row["ac_abf_p"]]
    n = sum(x is not None for x in p_list)


    for i, p in enumerate(p_list):
        if not p:
            p_list[i] = 1
    
    p_list = ["{:f}".format(p) for p in p_list]

    trend_list = [row["var_hr_trend"], row["var_rr_trend"], row["var_abf_trend"], row["ac_hr_trend"], row["ac_rr_trend"], row["ac_abf_trend"]]
    tau_list = [row["var_hr_tau"], row["var_rr_tau"], row["var_abf_tau"], row["ac_hr_tau"], row["ac_rr_tau"], row["ac_abf_tau"]]
    temp = np.array([[measure, p, tau, trend] for measure, p, tau, trend in zip(measure_list, p_list, tau_list, trend_list)])

    temp = temp[temp[:, 1].argsort()]

    measure_sorted = temp[:,0]
    p_sorted = temp[:,1]
    tau_sorted = temp[:,2]
    trend_sorted = temp[:,3]
    alphas = [float(alpha/(n-rank+1)) for rank in range(1,n+1)]

    failed = []

    for i, measure, p, trend, alpha in zip(range(1, n+1), measure_sorted, p_sorted, trend_sorted, alphas):
        if float(p) < alpha and trend == "increasing":
            print(p,alpha,trend)
            cohort_1_details_copy.at[idx,measure] = 1
            passed.append((cohort_1_details.loc[idx]["Project ID"], measure, p, trend))
        else:
            cohort_1_details_copy.at[idx,measure] = 0
    

0.002738 0.0125 increasing
0.006098 0.016666666666666666 increasing
0.006461 0.016666666666666666 increasing
0.000021 0.008333333333333333 increasing
0.000171 0.0125 increasing
0.000673 0.016666666666666666 increasing
0.002288 0.025 increasing
0.022835 0.05 increasing
0.000000 0.0125 increasing
0.007367 0.025 increasing
0.002849 0.0125 increasing
0.000035 0.008333333333333333 increasing
0.000092 0.01 increasing
0.001384 0.008333333333333333 increasing
0.000016 0.01 increasing
0.000000 0.008333333333333333 increasing
0.000000 0.008333333333333333 increasing
0.000000 0.01 increasing
0.008467 0.016666666666666666 increasing
0.010898 0.025 increasing
0.000003 0.01 increasing
0.000024 0.0125 increasing
0.000001 0.0125 increasing
0.002090 0.0125 increasing
0.000005 0.008333333333333333 increasing
0.000014 0.01 increasing
0.012163 0.0125 increasing
0.000268 0.0125 increasing
0.003392 0.025 increasing
0.000113 0.008333333333333333 increasing
0.000147 0.01 increasing
0.003212 0.0125 increasing


In [20]:
cohort_1_details_copy

,Unnamed: 0,Project ID,birth_deid_date,death_deid_date,sex,icu_visit,icu_start_deid_dttm,icu_end_deid_dttm,failed_extubation_flag,failed_extubation_deid_date,...,var_abf_trend,ac_hr_trend,ac_rr_trend,ac_abf_trend,var_hr_sig_flag,var_rr_sig_flag,var_abf_sig_flag,ac_hr_sig_flag,ac_rr_sig_flag,ac_abf_sig_flag
0,85,KH0067,2003-01-22 00:00:00,NaN,Female,2,2016-10-13 22:30:00,2016-12-03 15:54:00,1.0,2016-10-16 10:30:00,...,no trend,increasing,decreasing,decreasing,1,1,0,0,0,0
1,165,JJ0136,2001-06-12 00:00:00,NaN,Female,2,2016-06-22 02:00:00,2016-08-22 18:08:00,1.0,2016-07-05 12:15:00,...,decreasing,decreasing,no trend,no trend,0,0,0,0,0,0
2,204,SZ0162,2006-01-09 00:00:00,NaN,Male,1,2016-04-12 15:30:00,2016-05-12 13:00:00,1.0,2016-04-28 16:00:00,...,None,increasing,no trend,None,0,0,None,1,0,None
3,239,PC0192,2004-06-22 00:00:00,NaN,Male,1,2016-12-22 11:53:00,2017-02-18 17:13:00,1.0,2017-01-10 18:00:00,...,increasing,increasing,decreasing,increasing,1,1,1,1,0,1
4,314,GP0251,2001-10-06 00:00:00,NaN,Male,1,2018-06-25 14:44:00,2018-07-06 16:07:00,1.0,2018-06-26 15:00:00,...,no trend,no trend,decreasing,decreasing,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,5424,VV3912,2018-08-23 00:00:00,NaN,Male,2,2018-11-13 14:32:00,2018-11-17 17:20:00,1.0,2018-11-14 18:00:00,...,None,no trend,None,None,0,None,None,0,None,None
116,5484,PL3964,2018-09-29 00:00:00,NaN,Male,1,2018-10-10 17:30:00,2018-11-17 13:06:00,1.0,2018-10-22 15:00:00,...,None,no trend,no trend,None,0,0,None,0,0,None
117,5496,RD3976,2018-10-14 00:00:00,NaN,Male,1,2018-10-16 05:45:00,2018-10-23 19:09:00,1.0,2018-10-19 11:00:00,...,None,no trend,no trend,None,1,0,None,0,0,None
118,5565,TV4037,2018-09-28 00:00:00,NaN,Male,1,2018-12-12 20:00:00,2018-12-18 16:17:00,1.0,2018-12-13 11:50:00,...,None,no trend,no trend,None,0,0,None,0,0,None


In [21]:
# cohort_1_details_copy.to_csv("test_redo.csv")

# COHORT 2

In [22]:
var_hr_c2_results = pd.read_csv("variance_results_c2_entire.csv", sep=",")
var_rr_c2_results = pd.read_csv("variance_rr_results_c2_pt1.csv", sep=",")
var_abf_c2_results = pd.read_csv("variance_abf_results_c2_pt1.csv", sep=",")

ac_hr_c2_results = pd.read_csv("autocorrelation_hr_results_c2_partial.csv", sep=",")
ac_rr_c2_results = pd.read_csv("autocorrelation_rr_results_c2_partial.csv", sep=",")
ac_abf_c2_results = pd.read_csv("autocorrelation_abf_results_c2_partial.csv", sep=",")

In [23]:
var_hr_c2_results_tuples = list(zip(var_hr_c2_results["Project ID"], var_hr_c2_results["visit_no"]))
# var_hr_c1_failed_tuples = list(zip(var_hr_c1_failed["Project ID"], var_hr_c1_failed["visit_no"]))

var_rr_c2_results_tuples = list(zip(var_rr_c2_results["Project ID"], var_rr_c2_results["visit_no"]))
# var_rr_c1_failed_tuples = list(zip(var_rr_c1_failed["Project ID"], var_rr_c1_failed["visit_no"]))

var_abf_c2_results_tuples = list(zip(var_abf_c2_results["Project ID"], var_abf_c2_results["visit_no"]))
# var_abf_c1_failed_tuples = list(zip(var_abf_c1_failed["Project ID"], var_abf_c1_failed["visit_no"]))

ac_hr_c2_results_tuples = list(zip(ac_hr_c2_results["Project ID"], ac_hr_c2_results["visit_no"]))
# ac_hr_c1_failed_tuples = list(zip(ac_hr_c1_failed["Project ID"], ac_hr_c1_failed["visit_no"]))

ac_rr_c2_results_tuples = list(zip(ac_rr_c2_results["Project ID"], ac_rr_c2_results["visit_no"]))
# ac_rr_c1_failed_tuples = list(zip(ac_rr_c1_failed["Project ID"], ac_rr_c1_failed["visit_no"]))

ac_abf_c2_results_tuples = list(zip(ac_abf_c2_results["Project ID"], ac_abf_c2_results["visit_no"]))
# ac_abf_c1_failed_tuples = list(zip(ac_abf_c1_failed["Project ID"], ac_abf_c1_failed["visit_no"]))

In [24]:
cohort_2_details["var_hr_p"] = None
cohort_2_details["var_rr_p"] = None
cohort_2_details["var_abf_p"] = None
cohort_2_details["ac_hr_p"] = None
cohort_2_details["ac_rr_p"] = None
cohort_2_details["ac_abf_p"] = None

cohort_2_details["var_hr_tau"] = None
cohort_2_details["var_rr_tau"] = None
cohort_2_details["var_abf_tau"] = None
cohort_2_details["ac_hr_tau"] = None
cohort_2_details["ac_rr_tau"] = None
cohort_2_details["ac_abf_tau"] = None

cohort_2_details["var_hr_trend"] = None
cohort_2_details["var_rr_trend"] = None
cohort_2_details["var_abf_trend"] = None
cohort_2_details["ac_hr_trend"] = None
cohort_2_details["ac_rr_trend"] = None
cohort_2_details["ac_abf_trend"] = None

In [25]:
for idx, project_id, visit_no in zip(cohort_2_details.index, cohort_2_details["Project ID"], cohort_2_details["icu_visit"]):
    if (project_id, visit_no) in var_hr_c2_results_tuples:
        x = var_hr_c2_results.loc[var_hr_c2_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_2_details["var_hr_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_2_details["var_hr_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_2_details["var_hr_trend"].loc[idx] = x["trend"].iloc[0]

for idx, project_id, visit_no in zip(cohort_2_details.index, cohort_2_details["Project ID"], cohort_2_details["icu_visit"]):
    if (project_id, visit_no) in var_rr_c2_results_tuples:
        x = var_rr_c2_results.loc[var_rr_c2_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_2_details["var_rr_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_2_details["var_rr_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_2_details["var_rr_trend"].loc[idx] = x["trend"].iloc[0]

for idx, project_id, visit_no in zip(cohort_2_details.index, cohort_2_details["Project ID"], cohort_2_details["icu_visit"]):
    if (project_id, visit_no) in var_abf_c2_results_tuples:
        x = var_abf_c2_results.loc[var_abf_c2_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_2_details["var_abf_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_2_details["var_abf_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_2_details["var_abf_trend"].loc[idx] = x["trend"].iloc[0]


/Users/lucindakhalil/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [26]:
for idx, project_id, visit_no in zip(cohort_2_details.index, cohort_2_details["Project ID"], cohort_2_details["icu_visit"]):
    if (project_id, visit_no) in ac_hr_c2_results_tuples:
        x = ac_hr_c2_results.loc[ac_hr_c2_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_2_details["ac_hr_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_2_details["ac_hr_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_2_details["ac_hr_trend"].loc[idx] = x["trend"].iloc[0]

for idx, project_id, visit_no in zip(cohort_2_details.index, cohort_2_details["Project ID"], cohort_2_details["icu_visit"]):
    if (project_id, visit_no) in ac_rr_c2_results_tuples:
        x = ac_rr_c2_results.loc[ac_rr_c2_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_2_details["ac_rr_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_2_details["ac_rr_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_2_details["ac_rr_trend"].loc[idx] = x["trend"].iloc[0]

for idx, project_id, visit_no in zip(cohort_2_details.index, cohort_2_details["Project ID"], cohort_2_details["icu_visit"]):
    if (project_id, visit_no) in ac_abf_c2_results_tuples:
        x = ac_abf_c2_results.loc[ac_abf_c2_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_2_details["ac_abf_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_2_details["ac_abf_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_2_details["ac_abf_trend"].loc[idx] = x["trend"].iloc[0]

In [27]:
cohort_2_details["var_hr_sig_flag"] = None
cohort_2_details["var_rr_sig_flag"] = None
cohort_2_details["var_abf_sig_flag"] = None
cohort_2_details["ac_hr_sig_flag"] = None
cohort_2_details["ac_rr_sig_flag"] = None
cohort_2_details["ac_abf_sig_flag"] = None

In [28]:
alpha = 0.05
passed = []

cohort_2_details_copy = cohort_2_details.copy()

for idx in cohort_2_details.index:
    row = cohort_2_details.iloc[idx]

    # temp_dict_p = {"var_hr":row["var_hr_p"], "var_rr":row["var_rr_p"], "var_abf":row["var_abf_p"], "ac_hr":row["ac_hr_p"], "ac_rr":row["ac_rr_p"], "ac_abf":row["ac_abf_p"]}

    measure_list = ["var_hr_sig_flag","var_rr_sig_flag","var_abf_sig_flag","ac_hr_sig_flag","ac_rr_sig_flag","ac_abf_sig_flag"]
    p_list = [row["var_hr_p"], row["var_rr_p"], row["var_abf_p"], row["ac_hr_p"], row["ac_rr_p"], row["ac_abf_p"]]
    n = sum(x is not None for x in p_list)


    for i, p in enumerate(p_list):
        if not p:
            p_list[i] = 1
    
    p_list = ["{:f}".format(p) for p in p_list]

    trend_list = [row["var_hr_trend"], row["var_rr_trend"], row["var_abf_trend"], row["ac_hr_trend"], row["ac_rr_trend"], row["ac_abf_trend"]]
    tau_list = [row["var_hr_tau"], row["var_rr_tau"], row["var_abf_tau"], row["ac_hr_tau"], row["ac_rr_tau"], row["ac_abf_tau"]]
    temp = np.array([[measure, p, tau, trend] for measure, p, tau, trend in zip(measure_list, p_list, tau_list, trend_list)])

    temp = temp[temp[:, 1].argsort()]

    measure_sorted = temp[:,0]
    p_sorted = temp[:,1]
    tau_sorted = temp[:,2]
    trend_sorted = temp[:,3]
    alphas = [float(alpha/(n-rank+1)) for rank in range(1,n+1)]

    failed = []

    for i, measure, p, trend, alpha in zip(range(1, n+1), measure_sorted, p_sorted, trend_sorted, alphas):
        if float(p) < alpha and trend == "increasing":
            print(p,alpha,trend)
            cohort_2_details_copy.at[idx,measure] = 1
            passed.append((cohort_2_details.loc[idx]["Project ID"], measure, p, trend))
        else:
            cohort_2_details_copy.at[idx,measure] = 0
    

0.000018 0.0125 increasing
0.000000 0.0125 increasing
0.000120 0.0125 increasing
0.002455 0.025 increasing
0.002013 0.008333333333333333 increasing
0.006128 0.01 increasing
0.000315 0.0125 increasing
0.000001 0.008333333333333333 increasing
0.000000 0.008333333333333333 increasing
0.000000 0.01 increasing
0.000144 0.016666666666666666 increasing
0.002381 0.0125 increasing
0.000002 0.008333333333333333 increasing
0.000116 0.01 increasing
0.005046 0.0125 increasing
0.000106 0.025 increasing
0.000437 0.0125 increasing
0.024606 0.025 increasing
0.000991 0.008333333333333333 increasing
0.007187 0.01 increasing
0.000006 0.0125 increasing
0.000003 0.0125 increasing
0.000038 0.016666666666666666 increasing
0.006902 0.025 increasing
0.000001 0.0125 increasing
0.000000 0.0125 increasing
0.000000 0.016666666666666666 increasing
0.000011 0.008333333333333333 increasing
0.009485 0.0125 increasing
0.000000 0.008333333333333333 increasing
0.005660 0.01 increasing
0.006586 0.0125 increasing
0.000179 0

In [29]:
cohort_2_details_copy

,Unnamed: 0,Unnamed: 0.1,Project ID,birth_deid_date,death_deid_date,sex,icu_visit,icu_start_deid_dttm,icu_end_deid_dttm,failed_extubation_flag,...,var_abf_trend,ac_hr_trend,ac_rr_trend,ac_abf_trend,var_hr_sig_flag,var_rr_sig_flag,var_abf_sig_flag,ac_hr_sig_flag,ac_rr_sig_flag,ac_abf_sig_flag
0,0,4183,FJ2998,2017-09-24,NaN,Male,1,2017-09-28 16:15:00,2017-10-13 23:35:00,NaN,...,no trend,no trend,no trend,no trend,0,0,0,0,0,0
1,1,1807,ZA1349,2016-03-02,NaN,Female,1,2016-03-02 22:15:00,2016-03-08 19:14:00,NaN,...,no trend,no trend,None,no trend,0,None,0,0,None,0
2,2,4188,HM3001,2017-09-20,NaN,Male,1,2017-09-23 16:00:00,2017-10-13 16:30:00,NaN,...,None,no trend,no trend,None,0,1,None,0,0,None
3,3,3996,QV2851,2017-07-28,NaN,Male,2,2017-07-30 08:00:00,2017-08-05 15:41:00,NaN,...,None,no trend,increasing,None,0,0,None,0,1,None
4,4,5489,QH3969,2018-10-12,NaN,Male,1,2018-10-13 15:00:00,2018-10-24 16:40:00,NaN,...,no trend,no trend,decreasing,no trend,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005,2005,299,JD0238,2000-01-20,NaN,Male,1,2018-06-10 14:11:00,2018-06-11 13:15:00,NaN,...,increasing,no trend,no trend,increasing,1,0,1,0,0,1
2006,2006,182,BM0148,2004-06-09,NaN,Male,1,2016-10-30 00:45:00,2016-11-03 18:30:00,NaN,...,no trend,no trend,no trend,decreasing,0,0,0,0,0,0
2007,2007,355,SQ0287,2003-01-11,NaN,Female,1,2018-07-20 16:53:00,2018-07-22 11:25:00,NaN,...,None,increasing,decreasing,None,1,1,0,1,None,None
2008,2008,183,CH0149,2003-05-07,NaN,Female,1,2017-07-05 18:10:00,2017-07-09 13:35:00,NaN,...,decreasing,no trend,no trend,no trend,0,0,0,0,0,0


In [30]:
# cohort_2_details_copy.to_csv("cohort_2_details_redo.csv")

# COHORT 3

In [31]:
var_hr_c3_results = pd.read_csv("variance_hr_results_c3_entire.csv", sep=",")
var_rr_c3_results = pd.read_csv("variance_rr_results_c3.csv", sep=",")
var_abf_c3_results = pd.read_csv("variance_abf_results_c3.csv", sep=",")

ac_hr_c3_results = pd.read_csv("autocorrelation_hr_results_c3.csv", sep=",")
ac_rr_c3_results = pd.read_csv("autocorrelation_rr_results_c3.csv", sep=",")
ac_abf_c3_results = pd.read_csv("autocorrelation_abf_results_c3.csv", sep=",")

In [32]:
var_hr_c3_results

,Unnamed: 0,Unnamed: 0.1,Project ID,visit_no,trend,p-value,tau
0,0,0,MF2141,2,increasing,3.182835e-09,0.439547
1,1,1,ME0730,2,decreasing,8.352024e-05,-0.316469
2,2,2,ML3075,2,increasing,4.534344e-04,0.448711
3,3,3,JK3168,2,no trend,1.646666e-01,0.036667
4,4,4,CE1820,2,no trend,1.555504e-01,0.098217
...,...,...,...,...,...,...,...
110,77,77,HO3405,2,no trend,2.680022e-01,0.286667
111,78,78,HH3535,2,increasing,1.059825e-03,0.148522
112,79,79,HX3033,2,decreasing,1.380634e-04,-0.382591
113,80,80,MR2536,2,no trend,1.133064e-01,-0.124516


In [33]:
var_hr_c3_results_tuples = list(zip(var_hr_c3_results["Project ID"], var_hr_c3_results["visit_no"]))
# var_hr_c1_failed_tuples = list(zip(var_hr_c1_failed["Project ID"], var_hr_c1_failed["visit_no"]))

var_rr_c3_results_tuples = list(zip(var_rr_c3_results["Project ID"], var_rr_c3_results["visit_no"]))
# var_rr_c1_failed_tuples = list(zip(var_rr_c1_failed["Project ID"], var_rr_c1_failed["visit_no"]))

var_abf_c3_results_tuples = list(zip(var_abf_c3_results["Project ID"], var_abf_c3_results["visit_no"]))
# var_abf_c1_failed_tuples = list(zip(var_abf_c1_failed["Project ID"], var_abf_c1_failed["visit_no"]))

ac_hr_c3_results_tuples = list(zip(ac_hr_c3_results["Project ID"], ac_hr_c3_results["visit_no"]))
# ac_hr_c1_failed_tuples = list(zip(ac_hr_c1_failed["Project ID"], ac_hr_c1_failed["visit_no"]))

ac_rr_c3_results_tuples = list(zip(ac_rr_c3_results["Project ID"], ac_rr_c3_results["visit_no"]))
# ac_rr_c1_failed_tuples = list(zip(ac_rr_c1_failed["Project ID"], ac_rr_c1_failed["visit_no"]))

ac_abf_c3_results_tuples = list(zip(ac_abf_c3_results["Project ID"], ac_abf_c3_results["visit_no"]))
# ac_abf_c1_failed_tuples = list(zip(ac_abf_c1_failed["Project ID"], ac_abf_c1_failed["visit_no"]))

In [34]:
cohort_3_details["var_hr_p"] = None
cohort_3_details["var_rr_p"] = None
cohort_3_details["var_abf_p"] = None
cohort_3_details["ac_hr_p"] = None
cohort_3_details["ac_rr_p"] = None
cohort_3_details["ac_abf_p"] = None

cohort_3_details["var_hr_tau"] = None
cohort_3_details["var_rr_tau"] = None
cohort_3_details["var_abf_tau"] = None
cohort_3_details["ac_hr_tau"] = None
cohort_3_details["ac_rr_tau"] = None
cohort_3_details["ac_abf_tau"] = None

cohort_3_details["var_hr_trend"] = None
cohort_3_details["var_rr_trend"] = None
cohort_3_details["var_abf_trend"] = None
cohort_3_details["ac_hr_trend"] = None
cohort_3_details["ac_rr_trend"] = None
cohort_3_details["ac_abf_trend"] = None

In [35]:
for idx, project_id, visit_no in zip(cohort_3_details.index, cohort_3_details["Project ID"], cohort_3_details["icu_visit"]):
    if (project_id, visit_no) in var_hr_c3_results_tuples:
        x = var_hr_c3_results.loc[var_hr_c3_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_3_details["var_hr_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_3_details["var_hr_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_3_details["var_hr_trend"].loc[idx] = x["trend"].iloc[0]

for idx, project_id, visit_no in zip(cohort_3_details.index, cohort_3_details["Project ID"], cohort_3_details["icu_visit"]):
    if (project_id, visit_no) in var_rr_c3_results_tuples:
        x = var_rr_c3_results.loc[var_rr_c3_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_3_details["var_rr_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_3_details["var_rr_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_3_details["var_rr_trend"].loc[idx] = x["trend"].iloc[0]

for idx, project_id, visit_no in zip(cohort_3_details.index, cohort_3_details["Project ID"], cohort_3_details["icu_visit"]):
    if (project_id, visit_no) in var_abf_c3_results_tuples:
        x = var_abf_c3_results.loc[var_abf_c3_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_3_details["var_abf_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_3_details["var_abf_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_3_details["var_abf_trend"].loc[idx] = x["trend"].iloc[0]

/Users/lucindakhalil/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [36]:
for idx, project_id, visit_no in zip(cohort_3_details.index, cohort_3_details["Project ID"], cohort_3_details["icu_visit"]):
    if (project_id, visit_no) in ac_hr_c3_results_tuples:
        x = ac_hr_c3_results.loc[ac_hr_c3_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_3_details["ac_hr_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_3_details["ac_hr_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_3_details["ac_hr_trend"].loc[idx] = x["trend"].iloc[0]

for idx, project_id, visit_no in zip(cohort_3_details.index, cohort_3_details["Project ID"], cohort_3_details["icu_visit"]):
    if (project_id, visit_no) in ac_rr_c3_results_tuples:
        x = ac_rr_c3_results.loc[ac_rr_c3_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_3_details["ac_rr_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_3_details["ac_rr_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_3_details["ac_rr_trend"].loc[idx] = x["trend"].iloc[0]

for idx, project_id, visit_no in zip(cohort_3_details.index, cohort_3_details["Project ID"], cohort_3_details["icu_visit"]):
    if (project_id, visit_no) in ac_abf_c3_results_tuples:
        x = ac_abf_c3_results.loc[ac_abf_c3_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_3_details["ac_abf_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_3_details["ac_abf_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_3_details["ac_abf_trend"].loc[idx] = x["trend"].iloc[0]

In [37]:
cohort_3_details["var_hr_sig_flag"] = None
cohort_3_details["var_rr_sig_flag"] = None
cohort_3_details["var_abf_sig_flag"] = None
cohort_3_details["ac_hr_sig_flag"] = None
cohort_3_details["ac_rr_sig_flag"] = None
cohort_3_details["ac_abf_sig_flag"] = None

In [38]:
alpha = 0.05
passed = []

cohort_3_details_copy = cohort_3_details.copy()

for idx in cohort_3_details.index:
    row = cohort_3_details.iloc[idx]

    # temp_dict_p = {"var_hr":row["var_hr_p"], "var_rr":row["var_rr_p"], "var_abf":row["var_abf_p"], "ac_hr":row["ac_hr_p"], "ac_rr":row["ac_rr_p"], "ac_abf":row["ac_abf_p"]}

    measure_list = ["var_hr_sig_flag","var_rr_sig_flag","var_abf_sig_flag","ac_hr_sig_flag","ac_rr_sig_flag","ac_abf_sig_flag"]
    p_list = [row["var_hr_p"], row["var_rr_p"], row["var_abf_p"], row["ac_hr_p"], row["ac_rr_p"], row["ac_abf_p"]]
    n = sum(x is not None for x in p_list)


    for i, p in enumerate(p_list):
        if not p:
            p_list[i] = 1
    
    p_list = ["{:f}".format(p) for p in p_list]

    trend_list = [row["var_hr_trend"], row["var_rr_trend"], row["var_abf_trend"], row["ac_hr_trend"], row["ac_rr_trend"], row["ac_abf_trend"]]
    tau_list = [row["var_hr_tau"], row["var_rr_tau"], row["var_abf_tau"], row["ac_hr_tau"], row["ac_rr_tau"], row["ac_abf_tau"]]
    temp = np.array([[measure, p, tau, trend] for measure, p, tau, trend in zip(measure_list, p_list, tau_list, trend_list)])

    temp = temp[temp[:, 1].argsort()]

    measure_sorted = temp[:,0]
    p_sorted = temp[:,1]
    tau_sorted = temp[:,2]
    trend_sorted = temp[:,3]
    alphas = [float(alpha/(n-rank+1)) for rank in range(1,n+1)]

    failed = []

    for i, measure, p, trend, alpha in zip(range(1, n+1), measure_sorted, p_sorted, trend_sorted, alphas):
        if float(p) < alpha and trend == "increasing":
            print(p,alpha,trend)
            cohort_3_details_copy.at[idx,measure] = 1
            passed.append((cohort_3_details.loc[idx]["Project ID"], measure, p, trend))
        else:
            cohort_3_details_copy.at[idx,measure] = 0
    

0.000074 0.008333333333333333 increasing
0.003452 0.01 increasing
0.003188 0.01 increasing
0.000001 0.0125 increasing
0.000297 0.0125 increasing
0.000127 0.008333333333333333 increasing
0.000428 0.016666666666666666 increasing
0.003225 0.016666666666666666 increasing
0.016906 0.025 increasing
0.003369 0.01 increasing
0.000000 0.008333333333333333 increasing
0.005589 0.008333333333333333 increasing
0.000000 0.008333333333333333 increasing
0.017567 0.025 increasing
0.003542 0.01 increasing
0.000000 0.008333333333333333 increasing
0.000249 0.01 increasing
0.003067 0.0125 increasing
0.000000 0.008333333333333333 increasing
0.002814 0.025 increasing
0.004064 0.008333333333333333 increasing
0.011334 0.0125 increasing
0.000041 0.0125 increasing
0.001407 0.016666666666666666 increasing
0.000007 0.01 increasing
0.000000 0.008333333333333333 increasing
0.000002 0.01 increasing
0.040424 0.05 increasing
0.000000 0.0125 increasing
0.004131 0.0125 increasing
0.000023 0.025 increasing
0.020056 0.05 i

In [39]:
# cohort_3_details_copy.to_csv("cohort_3_details_redo.csv")

# COHORT 4

In [40]:
var_hr_c4_results = pd.read_csv("variance_hr_results_c4.csv", sep=",")
var_rr_c4_results = pd.read_csv("variance_rr_results_c4.csv", sep=",")
var_abf_c4_results = pd.read_csv("variance_abf_results_c4.csv", sep=",")

ac_hr_c4_results = pd.read_csv("autocorrelation_hr_results_c4.csv", sep=",")
ac_rr_c4_results = pd.read_csv("autocorrelation_rr_results_c4.csv", sep=",")
ac_abf_c4_results = pd.read_csv("autocorrelation_abf_results_c4.csv", sep=",")


In [41]:
var_hr_c4_results

,Unnamed: 0,Project ID,visit_no,trend,p-value,tau
0,0,GV3647,1,increasing,0.002919,0.380915
1,1,FM2255,1,no trend,0.781516,0.046361
2,2,JV3994,1,no trend,0.118536,-0.129634
3,3,DA1324,1,increasing,0.036867,0.433869
4,4,XZ2214,1,decreasing,0.000027,-0.630543
5,5,BY1872,1,no trend,0.589410,0.118021
6,6,WR2446,1,increasing,0.000014,0.778309
7,7,WA0589,1,increasing,0.010224,0.509378
8,8,TM2287,2,no trend,0.448736,-0.137229
9,9,QF1860,2,no trend,0.317779,-0.147676


In [42]:
var_hr_c4_results_tuples = list(zip(var_hr_c4_results["Project ID"], var_hr_c4_results["visit_no"]))
# var_hr_c1_failed_tuples = list(zip(var_hr_c1_failed["Project ID"], var_hr_c1_failed["visit_no"]))

var_rr_c4_results_tuples = list(zip(var_rr_c4_results["Project ID"], var_rr_c4_results["visit_no"]))
# var_rr_c1_failed_tuples = list(zip(var_rr_c1_failed["Project ID"], var_rr_c1_failed["visit_no"]))

var_abf_c4_results_tuples = list(zip(var_abf_c4_results["Project ID"], var_abf_c4_results["visit_no"]))
# var_abf_c1_failed_tuples = list(zip(var_abf_c1_failed["Project ID"], var_abf_c1_failed["visit_no"]))

ac_hr_c4_results_tuples = list(zip(ac_hr_c4_results["Project ID"], ac_hr_c4_results["visit_no"]))
# ac_hr_c1_failed_tuples = list(zip(ac_hr_c1_failed["Project ID"], ac_hr_c1_failed["visit_no"]))

ac_rr_c4_results_tuples = list(zip(ac_rr_c4_results["Project ID"], ac_rr_c4_results["visit_no"]))
# ac_rr_c1_failed_tuples = list(zip(ac_rr_c1_failed["Project ID"], ac_rr_c1_failed["visit_no"]))

ac_abf_c4_results_tuples = list(zip(ac_abf_c4_results["Project ID"], ac_abf_c4_results["visit_no"]))
# ac_abf_c1_failed_tuples = list(zip(ac_abf_c1_failed["Project ID"], ac_abf_c1_failed["visit_no"]))

In [43]:
ac_hr_c4_results_tuples

[('GV3647', 1.0),
 ('FM2255', 1.0),
 ('JV3994', 1.0),
 ('DA1324', 1.0),
 ('XZ2214', 1.0),
 ('BY1872', 1.0),
 ('WR2446', 1.0),
 ('WA0589', 1.0),
 ('TM2287', 2.0),
 ('QF1860', 2.0),
 ('AH3026', 4.0)]

In [44]:
cohort_4_details["var_hr_p"] = None
cohort_4_details["var_rr_p"] = None
cohort_4_details["var_abf_p"] = None
cohort_4_details["ac_hr_p"] = None
cohort_4_details["ac_rr_p"] = None
cohort_4_details["ac_abf_p"] = None

cohort_4_details["var_hr_tau"] = None
cohort_4_details["var_rr_tau"] = None
cohort_4_details["var_abf_tau"] = None
cohort_4_details["ac_hr_tau"] = None
cohort_4_details["ac_rr_tau"] = None
cohort_4_details["ac_abf_tau"] = None

cohort_4_details["var_hr_trend"] = None
cohort_4_details["var_rr_trend"] = None
cohort_4_details["var_abf_trend"] = None
cohort_4_details["ac_hr_trend"] = None
cohort_4_details["ac_rr_trend"] = None
cohort_4_details["ac_abf_trend"] = None

In [45]:
for idx, project_id, visit_no in zip(cohort_4_details.index, cohort_4_details["Project ID"], cohort_3_details["icu_visit"]):
    if (project_id, visit_no) in var_hr_c4_results_tuples:
        print((project_id, visit_no))
    else:
        print(f"not {(project_id, visit_no)}")

not ('LP0324', 2)
not ('TK0564', 1)
('WA0589', 1)
not ('HP0630', 1)
not ('XP0759', 1)
not ('JL0979', 1)
not ('UU1119', 2)
not ('CN1142', 1)
('DA1324', 1)
not ('EK1591', 1)
not ('QF1860', 1)
('BY1872', 1)
not ('TH2181', 1)
('XZ2214', 1)
('FM2255', 1)
not ('TM2287', 1)
not ('TF2429', 1)
not ('WR2446', 2)
not ('LC2708', 3)
not ('WH2753', 1)
not ('KP2971', 1)
not ('AH3026', 1)
not ('HH3535', 1)
('GV3647', 1)
not ('FD3688', 1)
not ('JV3994', 2)


In [46]:
var_hr_c4_results_tuples

[('GV3647', 1),
 ('FM2255', 1),
 ('JV3994', 1),
 ('DA1324', 1),
 ('XZ2214', 1),
 ('BY1872', 1),
 ('WR2446', 1),
 ('WA0589', 1),
 ('TM2287', 2),
 ('QF1860', 2),
 ('AH3026', 4)]

In [47]:
for idx, project_id, visit_no in zip(cohort_4_details.index, cohort_4_details["Project ID"], cohort_4_details["icu_visit"]):
    if (project_id, visit_no) in var_hr_c4_results_tuples:
        x = var_hr_c4_results.loc[var_hr_c4_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_4_details["var_hr_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_4_details["var_hr_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_4_details["var_hr_trend"].loc[idx] = x["trend"].iloc[0]

for idx, project_id, visit_no in zip(cohort_4_details.index, cohort_4_details["Project ID"], cohort_4_details["icu_visit"]):
    if (project_id, visit_no) in var_rr_c4_results_tuples:
        x = var_rr_c4_results.loc[var_rr_c4_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_4_details["var_rr_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_4_details["var_rr_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_4_details["var_rr_trend"].loc[idx] = x["trend"].iloc[0]

for idx, project_id, visit_no in zip(cohort_4_details.index, cohort_4_details["Project ID"], cohort_4_details["icu_visit"]):
    if (project_id, visit_no) in var_abf_c4_results_tuples:
        x = var_abf_c4_results.loc[var_abf_c4_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_4_details["var_abf_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_4_details["var_abf_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_4_details["var_abf_trend"].loc[idx] = x["trend"].iloc[0]

/Users/lucindakhalil/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [48]:
x

,Unnamed: 0,Project ID,visit_no,trend,p-value,tau
2,2,JV3994,1,no trend,0.841159,-0.022275


In [49]:
for idx, project_id, visit_no in zip(cohort_4_details.index, cohort_4_details["Project ID"], cohort_4_details["icu_visit"]):
    if (project_id, visit_no) in ac_hr_c4_results_tuples:
        x = ac_hr_c4_results.loc[ac_hr_c4_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_4_details["ac_hr_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_4_details["ac_hr_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_4_details["ac_hr_trend"].loc[idx] = x["trend"].iloc[0]

for idx, project_id, visit_no in zip(cohort_4_details.index, cohort_4_details["Project ID"], cohort_4_details["icu_visit"]):
    if (project_id, visit_no) in ac_rr_c4_results_tuples:
        x = ac_rr_c4_results.loc[ac_rr_c4_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_4_details["ac_rr_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_4_details["ac_rr_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_4_details["ac_rr_trend"].loc[idx] = x["trend"].iloc[0]

for idx, project_id, visit_no in zip(cohort_4_details.index, cohort_4_details["Project ID"], cohort_4_details["icu_visit"]):
    if (project_id, visit_no) in ac_abf_c4_results_tuples:
        x = ac_abf_c4_results.loc[ac_abf_c4_results["Project ID"] == project_id]
        x = x.loc[x["visit_no"] == int(visit_no)]
        cohort_4_details["ac_abf_tau"].loc[idx] = x["tau"].iloc[0]
        cohort_4_details["ac_abf_p"].loc[idx] = x["p-value"].iloc[0]
        cohort_4_details["ac_abf_trend"].loc[idx] = x["trend"].iloc[0]

/Users/lucindakhalil/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [50]:
cohort_4_details["var_hr_sig_flag"] = None
cohort_4_details["var_rr_sig_flag"] = None
cohort_4_details["var_abf_sig_flag"] = None
cohort_4_details["ac_hr_sig_flag"] = None
cohort_4_details["ac_rr_sig_flag"] = None
cohort_4_details["ac_abf_sig_flag"] = None

In [51]:
cohort_4_details

,Unnamed: 0,Project ID,birth_deid_date,death_deid_date,sex,icu_visit,icu_start_deid_dttm,icu_end_deid_dttm,failed_extubation_flag,failed_extubation_deid_date,...,var_abf_trend,ac_hr_trend,ac_rr_trend,ac_abf_trend,var_hr_sig_flag,var_rr_sig_flag,var_abf_sig_flag,ac_hr_sig_flag,ac_rr_sig_flag,ac_abf_sig_flag
0,400,LP0324,2009-07-23 00:00:00,2017-09-18 00:00:00,Female,1,2016-09-17 15:20:00,2016-09-24 14:30:00,1.0,2016-09-19 12:00:00,...,None,None,None,None,None,None,None,None,None,None
1,696,TK0564,2012-11-20 00:00:00,2017-10-31 00:00:00,Female,1,2017-09-22 08:30:00,2017-10-31 10:22:00,1.0,2017-09-23 07:00:00,...,None,None,None,None,None,None,None,None,None,None
2,728,WA0589,2011-10-27 00:00:00,2017-05-15 00:00:00,Male,1,2017-05-08 17:30:00,2017-05-15 19:45:00,1.0,2017-05-09 07:00:00,...,no trend,no trend,increasing,increasing,None,None,None,None,None,None
3,778,HP0630,2000-08-12 00:00:00,2017-06-03 00:00:00,Male,1,2017-04-08 20:10:00,2017-06-03 16:53:00,1.0,2017-04-10 21:00:00,...,None,None,None,None,None,None,None,None,None,None
4,948,XP0759,2013-03-24 00:00:00,2018-01-15 00:00:00,Male,2,2017-07-11 13:00:00,2017-07-24 14:55:00,1.0,2017-07-14 10:00:00,...,None,None,None,None,None,None,None,None,None,None
5,1284,JL0979,2015-08-15 00:00:00,2018-04-11 00:00:00,Male,2,2018-03-06 21:00:00,2018-04-10 09:53:00,1.0,2018-03-21 12:25:00,...,None,None,None,None,None,None,None,None,None,None
6,1467,UU1119,2014-09-28 00:00:00,2018-05-12 00:00:00,Male,1,2018-05-09 18:44:00,2018-05-12 18:47:00,1.0,2018-05-11 10:10:00,...,None,None,None,None,None,None,None,None,None,None
7,1505,CN1142,2011-06-03 00:00:00,2016-07-18 00:00:00,Male,2,2016-05-14 16:48:00,2016-05-25 16:00:00,1.0,2016-05-16 12:00:00,...,None,None,None,None,None,None,None,None,None,None
8,1777,DA1324,2013-10-05 00:00:00,2016-10-28 00:00:00,Female,1,2016-10-06 06:17:00,2016-10-28 22:30:00,1.0,2016-10-23 19:45:00,...,no trend,no trend,increasing,increasing,None,None,None,None,None,None
9,2189,EK1591,2016-05-16 00:00:00,2016-10-22 00:00:00,Male,1,2016-05-18 17:01:00,2016-08-23 11:30:00,1.0,2016-06-18 01:00:00,...,None,None,None,None,None,None,None,None,None,None


In [52]:
alpha = 0.05
passed = []

cohort_4_details_copy = cohort_4_details.copy()

for idx in cohort_4_details.index:
    row = cohort_4_details.iloc[idx]

    # temp_dict_p = {"var_hr":row["var_hr_p"], "var_rr":row["var_rr_p"], "var_abf":row["var_abf_p"], "ac_hr":row["ac_hr_p"], "ac_rr":row["ac_rr_p"], "ac_abf":row["ac_abf_p"]}

    measure_list = ["var_hr_sig_flag","var_rr_sig_flag","var_abf_sig_flag","ac_hr_sig_flag","ac_rr_sig_flag","ac_abf_sig_flag"]
    p_list = [row["var_hr_p"], row["var_rr_p"], row["var_abf_p"], row["ac_hr_p"], row["ac_rr_p"], row["ac_abf_p"]]
    n = sum(x is not None for x in p_list)


    for i, p in enumerate(p_list):
        if not p:
            p_list[i] = 1
    
    p_list = ["{:f}".format(p) for p in p_list]

    trend_list = [row["var_hr_trend"], row["var_rr_trend"], row["var_abf_trend"], row["ac_hr_trend"], row["ac_rr_trend"], row["ac_abf_trend"]]
    tau_list = [row["var_hr_tau"], row["var_rr_tau"], row["var_abf_tau"], row["ac_hr_tau"], row["ac_rr_tau"], row["ac_abf_tau"]]
    temp = np.array([[measure, p, tau, trend] for measure, p, tau, trend in zip(measure_list, p_list, tau_list, trend_list)])

    temp = temp[temp[:, 1].argsort()]

    measure_sorted = temp[:,0]
    p_sorted = temp[:,1]
    tau_sorted = temp[:,2]
    trend_sorted = temp[:,3]
    alphas = [float(alpha/(n-rank+1)) for rank in range(1,n+1)]

    failed = []

    for i, measure, p, trend, alpha in zip(range(1, n+1), measure_sorted, p_sorted, trend_sorted, alphas):
        if float(p) < alpha and trend == "increasing":
            print(p,alpha,trend)
            cohort_4_details_copy.at[idx,measure] = 1
            passed.append((cohort_4_details.loc[idx]["Project ID"], measure, p, trend))
        else:
            print(p,alpha,trend)
            cohort_4_details_copy.at[idx,measure] = 0

0.001985 0.008333333333333333 increasing
0.010224 0.01 increasing
0.037538 0.0125 increasing
0.098593 0.016666666666666666 no trend
0.127833 0.025 no trend
0.849391 0.05 no trend
0.000000 0.008333333333333333 increasing
0.000264 0.01 increasing
0.002658 0.0125 decreasing
0.036867 0.016666666666666666 increasing
0.116312 0.025 no trend
0.643625 0.05 no trend
0.002631 0.008333333333333333 increasing
0.177685 0.01 no trend
0.317779 0.0125 no trend
0.535599 0.016666666666666666 no trend
0.738177 0.025 no trend
0.821008 0.05 no trend
0.000000 0.008333333333333333 increasing
0.000003 0.01 increasing
0.088433 0.0125 no trend
0.486781 0.016666666666666666 no trend
0.589410 0.025 no trend
0.699931 0.05 no trend
0.000027 0.0125 decreasing
0.123657 0.016666666666666666 no trend
0.176641 0.025 no trend
0.728637 0.05 no trend
0.000096 0.008333333333333333 increasing
0.000219 0.01 increasing
0.042696 0.0125 increasing
0.588643 0.016666666666666666 no trend
0.589200 0.025 no trend
0.781516 0.05 no tr

In [53]:
# cohort_4_details_copy.to_csv("cohort_4_details_redo.csv")

In [54]:
cohort_4_details_copy["ac_abf_sig_flag"].value_counts()

1    5
0    4
Name: ac_abf_sig_flag, dtype: int64

In [82]:
cohort_2_details_copy.groupby("icu_ward", dropna=False).agg({'var_hr_sig_flag': ['sum'], 'var_rr_sig_flag': ['sum'], 'var_abf_sig_flag': ['sum'], 'ac_hr_sig_flag': ['sum'], 'ac_rr_sig_flag': ['sum'], 'ac_abf_sig_flag': ['sum']})

,var_hr_sig_flag,var_rr_sig_flag,var_abf_sig_flag,ac_hr_sig_flag,ac_rr_sig_flag,ac_abf_sig_flag
,sum,sum,sum,sum,sum,sum
icu_ward,,,,,,
FLAMI,231,221,166,168,115,177
NICU,60,61,9,51,39,12
PICU,122,106,45,87,83,33


s-test


In [83]:
cohort_2_details_copy

,Unnamed: 0,Unnamed: 0.1,Project ID,birth_deid_date,death_deid_date,sex,icu_visit,icu_start_deid_dttm,icu_end_deid_dttm,failed_extubation_flag,...,var_abf_trend,ac_hr_trend,ac_rr_trend,ac_abf_trend,var_hr_sig_flag,var_rr_sig_flag,var_abf_sig_flag,ac_hr_sig_flag,ac_rr_sig_flag,ac_abf_sig_flag
0,0,4183,FJ2998,2017-09-24,NaN,Male,1,2017-09-28 16:15:00,2017-10-13 23:35:00,NaN,...,no trend,no trend,no trend,no trend,0,0,0,0,0,0
1,1,1807,ZA1349,2016-03-02,NaN,Female,1,2016-03-02 22:15:00,2016-03-08 19:14:00,NaN,...,no trend,no trend,None,no trend,0,None,0,0,None,0
2,2,4188,HM3001,2017-09-20,NaN,Male,1,2017-09-23 16:00:00,2017-10-13 16:30:00,NaN,...,None,no trend,no trend,None,0,1,None,0,0,None
3,3,3996,QV2851,2017-07-28,NaN,Male,2,2017-07-30 08:00:00,2017-08-05 15:41:00,NaN,...,None,no trend,increasing,None,0,0,None,0,1,None
4,4,5489,QH3969,2018-10-12,NaN,Male,1,2018-10-13 15:00:00,2018-10-24 16:40:00,NaN,...,no trend,no trend,decreasing,no trend,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005,2005,299,JD0238,2000-01-20,NaN,Male,1,2018-06-10 14:11:00,2018-06-11 13:15:00,NaN,...,increasing,no trend,no trend,increasing,1,0,1,0,0,1
2006,2006,182,BM0148,2004-06-09,NaN,Male,1,2016-10-30 00:45:00,2016-11-03 18:30:00,NaN,...,no trend,no trend,no trend,decreasing,0,0,0,0,0,0
2007,2007,355,SQ0287,2003-01-11,NaN,Female,1,2018-07-20 16:53:00,2018-07-22 11:25:00,NaN,...,None,increasing,decreasing,None,1,1,0,1,None,None
2008,2008,183,CH0149,2003-05-07,NaN,Female,1,2017-07-05 18:10:00,2017-07-09 13:35:00,NaN,...,decreasing,no trend,no trend,no trend,0,0,0,0,0,0
